### mysql과 python의 연동
1. mysql과 연동을 할 수 있는 기능을 가진 라이브러리 설치 
2. 라이브러리 로드 
3. 어떠한 DB server에 접속할 것인가?
    - 서버의 정보를 등록 
        - 서버의 주소 
        - 서버의 포트번호
        - 서버의 아이디
        - 서버의 패스워드
        - 사용할 데이터베이스의 이름
4. DB서버와 Python 사이에 가상의 공간(커서)을 생성
5. sql 쿼리문을 이용하여 가상 공간에 질의를 보낸다. 
    - 데이터의 변경이 존재하는 쿼리문
        - 질의를 보낸다. 
        - 데이터베이스 서버의 데이터를 변경(커밋)
        - 데이터베이스 서버와의 연결을 종료
    - 데이터를 조회하는 쿼리문
        - 질의를 보낸다 
        - 질의에 대한 결과물을 받아온다. 
        - 데이터베이스 서버와의 연결을 종료

In [ ]:
# 라이브러리 설치 
!pip install pymysql

In [ ]:
# 라이브러리 로드 
import pymysql

In [ ]:
# mysql 연결시 cryptography 에러 발생시 
!pip install cryptography

In [ ]:
# DB 서버와 연결
_db = pymysql.connect(
    host = 'localhost', 
    port = 3306, 
    user = 'root', 
    password = '1234', 
    db = 'ezen'
)

In [ ]:
# 가상 공간(Cursor) 생성
# 기본값을 사용
cursor = _db.cursor()
# 옵션을 변경
cursor2 = _db.cursor(pymysql.cursors.DictCursor)

In [ ]:
# sql 쿼리문 작성
query = "select * from `user`"
# 가상 공간에 질의를 보낸다 (execute())
cursor.execute(query)
cursor2.execute(query)

In [ ]:
# 가상공간에서 질의에 대한 답변을 받아온다.(fetchall())
db_res = cursor.fetchall()
db_res2 = cursor2.fetchall()

In [ ]:
# 기본 커서에서 불러온 데이터
print(db_res)

In [ ]:
print(db_res2)

In [ ]:
import pandas as pd

In [ ]:
df = pd.DataFrame(db_res)
df2 = pd.DataFrame(db_res2)

In [ ]:
df

In [ ]:
df2

In [ ]:
# 데이터를 추가
insert_query = """
    insert into 
    `user` 
    values 
    (
        'test4', 
        '0000', 
        'choi'
    )
"""
cursor2.execute(insert_query)

In [ ]:
# 데이터 확인 
cursor2.execute(query)
cursor2.fetchall()

In [ ]:
# DB 서버에 가상공간의 데이터를 대입(연동)
_db.commit()

In [ ]:
# 수정 쿼리문 작성 
# where 절이 존재하지 않기때문에 모든 password가 변경
update_query = """
    update `user` 
    set 
    `password` = '0000'
"""
# cursor2.execute(update_query)

In [ ]:
cursor2.execute(query)
cursor2.fetchall()

In [ ]:
# _db.close()

In [ ]:
# 외부에서 입력된 데이터를 가지고 쿼리문과 질의를 보낸다. 
# 패스워드 변경 
input_id = input("아이디를 입력하시오")
input_pass = input("변경할 패스워드 값을 입력하시오")
# print(input_id, input_pass)

update_query = f"""
    update `user`
    set 
    `password` = '{input_pass}'
    where `id` = '{input_id}'
"""
# print(update_query)
cursor2.execute(update_query)

In [ ]:
input_id = input("아이디를 입력하시오")
input_pass = input("변경할 패스워드 값을 입력하시오")
update_query2 = """
    update `user`
    set 
    `password` = %s  
    where `id` = %s
"""
cursor2.execute(update_query2, [input_pass, input_id])

In [ ]:
_db.commit()

### mysql연동하고 쿼리문을 입력시 결과물을 되돌려주는 Class 생성
1. 생성자 함수 : class가 생성될때 자동으로 실행되는 함수 
    - DB 서버의 정보를 등록 (변수에 저장)
2. sql_query() 함수 생성
    - 매개변수 2개 사용
        - _sql : 쿼리문을 입력 받는 부분
        - *_value : 인자의 개수가 가변인 경우의 매개변수를 생성 (%s에 들어갈 데이터 값)
    - DB 서버에 접속
    - 가상 공간(Cursor) 생성 : DictCursor를 이용하여 생성
    - 가상 공간에 질의 
    - 조건문을 사용
        - 질의가 select라면
            - 질의를 보낸 후 결과 값을 되돌려주는 코드 작성
        - 질의가 select가 아니라면
            - 질의를 보낸 후 DB에 동기화
    - DB 서버와의 연결을 종료


In [ ]:
class MyDB:
    # 생성자 함수 
    def __init__(
            self, 
            _host = 'localhost', 
            _port = 3306, 
            _user = 'root', 
            _password = '1234', 
            _db = 'ezen'):
        # 매개변수를 통해 입력된 인자값을 변수에 저장 
        self.host = _host
        self.port = _port
        self.user = _user
        self.password = _password
        self.db = _db
    # sql_query() 함수 생성
    def sql_query(self, _sql, *_values):
        # DB 서버와의 연결 
        _db = pymysql.connect(
            host = self.host, 
            port = self.port, 
            user = self.user, 
            password = self.password, 
            db = self.db
        )
        # 가상 공간 생성
        cursor = _db.cursor(pymysql.cursors.DictCursor)
        try:
            # 질의 
            cursor.execute(_sql, _values)
        except Exception as e:
            print('Error')
            _db.close()
            return e
        # 질의가 select문이라면?
        # if _sql.lower().split()[0] == 'select':
        # if _sql.strip().lower().startswith('select'):
        if _sql.strip().lower()[:6] == 'select':
            # 가상 공간에서 결과 값을 로드 
            result = cursor.fetchall()
        else:
            # DB 서버와 동기화 
            _db.commit()
            result = "Query OK"
        # DB 서버와의 연결을 종료
        _db.close()
        # 결과값을 되돌려준다. 
        return result
        

In [ ]:
# class를 이용하여 DB 서버와의 연결
db1 = MyDB()

In [ ]:
# MyDB class 안에 있는 함수를 호출 
query = """
    select 
    * 
    from 
    `user`
"""

db1.sql_query(query)

In [ ]:
query2 = """
    insert into 
    `user` 
    values 
    (%s, %s, %s)
"""
input_id = input('아이디를 입력하시오')
input_pass = input('비밀번호를 입력하시오')
input_name = input("이름을 입력하시오")

db1.sql_query(query2, input_id, input_pass, input_name)